# Bokeh: An Interactive Data Visualization Library in CodeBook

* Ref: [https://developers.refinitiv.com/en/article-catalog/article/bokeh--an-interactive-data-visualization-library-in-codebook](https://developers.refinitiv.com/en/article-catalog/article/bokeh--an-interactive-data-visualization-library-in-codebook)

In [1]:
# import eikon as ek
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, CDSView, BooleanFilter, HoverTool, FactorRange
from bokeh.transform import factor_cmap, cumsum
from bokeh.layouts import row, column, gridplot
from bokeh.models.widgets import Tabs, Panel
import pandas as pd
from math import pi
import datetime
from IPython.display import display

# ek.set_app_key("DEFAULT_CODE_BOOK_APP_KEY") # set your App Key
output_notebook()

Loading BokehJS ...

### Line Chart

In [9]:
# df1 = ek.get_timeseries(
#     "IBM.N",
#     fields=["CLOSE"],
#     start_date=datetime.timedelta(days=-365),
#     end_date=datetime.datetime.now(),
#     interval="daily"
# )
import yfinance as yf

df1 = yf.download(
    tickers="IBM",
    start="2021-01-01",
    end="2021-12-31",
    progress=False, # 不顯示進度條
)
display(df1.head())

figure1 = figure(
    title="Daily Close Price (IBM)",
    x_axis_type="datetime",
    x_axis_label="Date",
)

figure1.line(
    x="Date",
    y="Close",
    source=df1,
    legend_label="Close"
)

figure1.add_tools(
    HoverTool(
        tooltips=[
            ("Date", "$x{%F}"),
            ("Price", "$y{"+f"0.00"+" a}")
        ],
        formatters={
            "$x": "datetime",
        },
    )
)

figure1.legend.location = "bottom_right"
show(figure1)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-01-04,120.315491,120.382408,117.629066,118.489487,111.470741,5417443
2021-01-05,119.512428,121.108986,119.130020,120.592735,113.449394,6395872
2021-01-06,121.319313,126.080307,121.147224,123.604210,116.282486,8322708
2021-01-07,124.321220,124.722755,122.619499,123.317398,116.012665,4714740
2021-01-08,122.915871,123.632889,121.395790,122.877632,115.598953,4891305


### Line Chart with Multiple Lines

In [10]:
df1["SMA_15"] = df1["Close"].rolling(15).mean()
display(df1.head())

figure2 = figure(
    title="Daily Close Price and SMA (IBM)",
    x_axis_type="datetime",
    x_axis_label="Date"
)

figure2.line(
    x="Date",
    y="Close",
    source=df1,
    legend_label="Close",
    name="CLOSE"
)

figure2.line(
    x="Date",
    y="SMA_15",
    source=df1,
    legend_label="SMA 15",
    line_dash="dashed",
    color="red",
    name="SMA 15"
)

figure2.add_tools(
    HoverTool(
        tooltips=[
            ("", "$name"),
            ("Date", "$x{%F}"),
            ("Price", "$y"+f"0.00"+" a}")
        ],
        formatters={
            "$x": "datetime"
        },
    )
)

figure2.legend.location = "bottom_right"
show(figure2)

,Open,High,Low,Close,Adj Close,Volume,SMA_15
Date,,,,,,,
2021-01-04,120.315491,120.382408,117.629066,118.489487,111.470741,5417443,NaN
2021-01-05,119.512428,121.108986,119.130020,120.592735,113.449394,6395872,NaN
2021-01-06,121.319313,126.080307,121.147224,123.604210,116.282486,8322708,NaN
2021-01-07,124.321220,124.722755,122.619499,123.317398,116.012665,4714740,NaN
2021-01-08,122.915871,123.632889,121.395790,122.877632,115.598953,4891305,NaN


### Candlestick Chart

In [11]:
# df2 = ek.get_timeseries(
#     "IBM.N",
#     start_date=datetime.timedelta(days=-90),
#     end_date=datetime.datetime.now(),
#     interval="daily"
# )
df2 = yf.download(
    tickers="IBM",
    start="2021-10-01",
    end="2021-12-31",
    progress=False, # 不顯示進度條
)
display(df2.head())

source = ColumnDataSource(data=df2)
hover = HoverTool(
    tooltips=[
        ("", "@Date{%F}"),
        ("O", "@Open{"+f"0.00"+" a}"),
        ("H", "@High{"+f"0.00"+" a}"),
        ("L", "@Low{"+f"0.00"+" a}"),
        ("C", "@Close{"+f"0.00"+" a}"),
        ("V", "@Volume{0}")
    ],
    formatters={
        "@Date": "datetime"
    },
    mode="mouse"
)

inc_b = source.data["Close"] >= source.data["Open"]
inc = CDSView(
    source=source,
    filters=[BooleanFilter(inc_b)]
)
print(inc_b)
print(inc)

dec_b = source.data["Open"] > source.data["Close"]
dec = CDSView(
    source=source,
    filters=[BooleanFilter(dec_b)]
)
print(dec_b)
print(dec)

w = 12 * 60 * 60 * 1000

figure3 = figure(
    title="OHLC Candlestick Chart (IBM)",
    x_axis_type="datetime",
    sizing_mode="stretch_width",
    height=400,
    x_axis_label="Date"
)

figure3.segment(
    source=source,
    x0="Date",
    x1="Date",
    y0="High",
    y1="Low",
    color="black"
)

figure3.vbar(
    source=source,
    view=inc,
    x="Date",
    width=w,
    top="Open",
    bottom="Close",
    fill_color="green",
    line_color="green"
)

figure3.vbar(
    source=source,
    view=dec,
    x="Date",
    width=w,
    top="Open",
    bottom="Close",
    fill_color="red",
    line_color="red"
)

figure3.add_tools(hover)
show(figure3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-10-01,134.799240,137.638626,134.196945,137.017212,133.603577,6907889
2021-10-04,136.462708,139.579346,136.089859,137.772461,134.340012,7689251
2021-10-05,138.384323,138.623322,136.367111,136.854691,133.445114,7297524
2021-10-06,136.214142,137.065002,134.694077,136.099426,132.708664,5573506
2021-10-07,136.453156,137.093689,135.305923,135.573608,132.195938,3999695


[ True  True False False False  True False False  True  True  True False
  True  True False False  True False False  True False  True False  True
 False  True  True False False False False False False False False False
  True  True  True  True False False False False  True  True  True  True
  True False False  True False  True  True  True  True  True  True  True
  True  True  True]
CDSView(id='2373', ...)
[False False  True  True  True False  True  True False False False  True
 False False  True  True False  True  True False  True False  True False
  True False False  True  True  True  True  True  True  True  True  True
 False False False False  True  True  True  True False False False False
 False  True  True False  True False False False False False False False
 False False False]
CDSView(id='2375', ...)


### Vertical Bar Chart

In [ ]:
quarter = pd.Timestamp(datetime.date.today()).quarter
quarter = quarter + 14
quarter = quarter * -1

df3, err = ek.get_data(
    instruments=["IBM.N"],
    fields=["TR.Revenue.Date", "TR.Revenue"],
    parameters={
        "SDate": 0,
        "EDate": quarter,
        "Period": "FQ0", 
        "Frq": "FQ"
    }
)
df3 = df3[::-1].reset_index().drop("index", axis=1)
display(df3.head())

df3["Date"] = pd.to_datetime(df3["Date"], format="%Y-%m-%d")
df3["Q"] = ("Q" + df3["Date"].dt.quarter.astype(str))
df3["Year"] = df3["Date"].dt.to_period("Y").astype(str)
df3["QY"] = df3["Date"].dt.to_period("Q").astype(str)
df3["Group"] = df3[["Year", "Q"]].apply(lambda x: (x[0], str(x[1])), axis=1)
display(df3.head())

palette = ["#0000ff", "#00ff00", "#ffff00", "#00ffff"]
years = ["Q1", "Q2", "Q3", "Q4"]

figure4 = figure(
    x_range=FactorRange(*list(df3["Group"])),
    title="IBM.N Revenue"
    width=800
)

figure4.vbar(
    x="Group",
    top="Revenue",
    width=0.9
    source=df3,
    fill_color=factor_map(
        "Group",
        palette=palette,
        factors=years,
        start=1,
        end=3
    )
)

figure4.add_tools(
    HoverTool(
        tooltips=[
            ("", "@QY"),
            ("", "@Revenue{0.00 a}")
        ],
        mode="mouse"
    )
)

figure4.left[0].formatter.use_scientific = False
show(figure)

### Pie Chart

In [ ]:
df4, err = ek.get_date(
    "0#.DJI",
    ["TR.IndexConstituentWeightPercent", "TR.CommonName"]
)

df4 = df4.rename(
    columns={
        "Weight percent": "Value",
        "Company Common Name": "Name"
    }
)
display(df4.head())

df4["angle"] = df4["Value"] / df4["Value"].sum() * 2 * pi
df4["color"] = [
    '#1f77b4', '#aec7e8', '#ff7f0e', '#ffbb78', '#2ca02c', '#98df8a', '#d62728', '#ff9896', '#9467bd', '#c5b0d5',
    '#8c564b', '#c49c94', '#e377c2', '#f7b6d2', '#7f7f7f', '#c7c7c7', '#bcbd22', '#dbdb8d', '#17becf', '#9edae5',
    '#0000ff', '#ff0000', '#ffff00', '#00ffff', '#ff00ff', '#F0F8FF', '#006B3C', '#00FF6F', '#392972', '#8F00FF'
]
dislpay(df4.head())

figure5 = figure(
    plot_height=800,
    plot_width=800,
    title="Dow Jone Industrials Constituents Weighting (.DJI)",
    toolbar_location=None,
    tools="hover",
    tooltips="@Name (@Instrument): @Value",
    x_range=(-0.5, 1.0)
)

figure5.wedge(
    x=0,
    y=1,
    radius=0.4,
    start_angle=cumsum("angle", include_zero=True),
    end_angle=cumsum("angle"),
    line_color="white",
    fill_color="collor",
    legend_field="Instrument",
    source=df4
)

figure5.axis.axis_label=None
figure5.axis.visible=False
figure.grid.grid_line_color=None

show(figure5)

### Row and Column layout

In [ ]:
row_layout = column(
    row(
        children=[figure1, figure2],
        figure3
    )
)

### Grid layout

In [ ]:
grid_layout = gridplot(
    [
        [figure1, figure2],
        [figure3, figure4]
    ],
    plot_width=500,
    plot_height=500
)

show(grid_layout)

### Panel and Tabs

In [ ]:
panel1 = Panel(
    child=grid_layout,
    title="IBM.N"
)
panel2 = Panel(
    child=figure5,
    title="DJI"
)
tabs_object = Tabs(tabs=[panel1, panel2])
show(tabs_object)